In [116]:
import pandas as pd

In [148]:
file_deg = '../../data/res/04_02_19/DEG-normalized.csv'
file_ann = '../../data/genome/annotations/genes.gff3'
file_output = '../../data/res/deg_sRNA.csv'
path_dat_promoters = '../../data/ss/GTP_promoters/'
path_dat_genes = '../../data/ss/GTP_genes/'
path_dat_tes = '../../data/ss/GTP_TEs_vMITEs/'


In [118]:
#get DEG list

In [119]:
df_deg = pd.read_csv(file_deg, index_col=False, sep=',', comment='#')
print(len(df_deg.index))
df_deg['gene'] = df_deg['gene'].str.split('gene:').str[1]
df_deg.drop("Unnamed: 0", axis=1, inplace=True)

3019


In [120]:
df_deg.sample(3)

,gene,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,C1.counts.txt,C2.counts.txt,C3.counts.txt,C4.counts.txt,T1.counts.txt,T2.counts.txt,T3.counts.txt,T4.counts.txt
1881,TraesCS5A02G308900,570.813371,0.683258,0.202392,3.375916,0.000736,0.022306,353.436700,381.722000,526.966131,490.489368,557.344445,818.865313,653.265827,784.417183
690,TraesCS1D02G361800,67.512071,1.394833,0.333821,4.178386,0.000029,0.002423,56.936611,20.875422,39.696130,31.712675,108.535497,125.554084,86.416267,70.369879
2410,TraesCS3A02G164700,49.012025,1.357499,0.426147,3.185519,0.001445,0.034186,29.005443,10.934745,39.696130,30.655585,82.134971,99.495689,34.978013,65.195623


In [121]:
df_deg = df_deg[['gene','log2FoldChange']]

In [122]:
#get genes positions

In [123]:
df_ann = pd.read_csv(file_ann, index_col=False, sep='\t', comment='#', header=None)
df_ann.columns = ['seqname' , 'source' , 'feature' , 'start' , 'end' , 'score' , 'strand' , 'frame' , 'attribute']
df_ann['gene'] = df_ann.attribute.str.split('gene:').str[1]
df_ann['gene'] = df_ann.gene.str.split(';').str[0]
print(len(df_ann.index))


120744


In [124]:
df_ann = df_ann[['gene','seqname','start','end']]

In [125]:
df_ann.sample(2)

,gene,seqname,start,end
118181,TraesCSU02G238900,Un,354963326,354964152
70784,ENSRNA050023979,5B,30846619,30846699


In [126]:
#merge DEG and positions

In [127]:
df_deg_pos = pd.merge(df_deg, df_ann, on='gene')

In [128]:
df_deg_pos.sample(2)

,gene,log2FoldChange,seqname,start,end
829,TraesCS3D02G394200,1.610921,3D,509228440,509231408
1354,TraesCS6A02G367000,-0.529017,6A,594405188,594408225


In [129]:
#promoters

In [130]:
df_dat_promoters = pd.read_csv(path_dat_promoters + 'diffexpr-results.0.1.csv', index_col=False, sep=',', comment='#')
df_dat_promoters['gene'] = df_dat_promoters.Gene.str.split('gene:').str[1]
df_dat_promoters['gene'] = df_dat_promoters.gene.str.split(';').str[0]
df_dat_promoters = df_dat_promoters[['gene','log2FoldChange']]
df_dat_promoters.rename(columns={'gene':'promoter','log2FoldChange':'log2FoldChange_promoter'}, inplace=True)
print(len(df_dat_promoters.index))

182


In [131]:
df_dat_promoters.sample(3)

,promoter,log2FoldChange_promoter
14,TraesCS3B02G206000,1.255864
79,TraesCS3D02G498700,0.801119
169,TraesCSU02G272400,1.004169


In [132]:
df_res = pd.merge(df_deg_pos, df_dat_promoters, left_on='gene', right_on='promoter', how='left')

In [133]:
df_res[~pd.isnull(df_res.promoter)]

,gene,log2FoldChange,seqname,start,end,promoter,log2FoldChange_promoter
1654,TraesCS2A02G474000,-0.571754,2A,715336822,715343560,TraesCS2A02G474000,0.838466


In [134]:
#genes

In [135]:
df_dat_genes = pd.read_csv(path_dat_genes + 'diffexpr-results.0.1.csv', index_col=False, sep=',', comment='#')
df_dat_genes['gene'] = df_dat_genes.Gene.str.split('gene:').str[1]
df_dat_genes['gene'] = df_dat_genes.gene.str.split(';').str[0]
df_dat_genes = df_dat_genes[['gene','log2FoldChange']]
df_dat_genes.rename(columns={'gene':'sRNA_gene','log2FoldChange':'log2FoldChange_gene'}, inplace=True)
print(len(df_dat_genes.index))

55


In [136]:
df_res = pd.merge(df_res, df_dat_genes, left_on='gene', right_on='sRNA_gene', how='left')
#df_res[(~pd.isnull(df_res.promoter)) | (~pd.isnull(df_res.sRNA_gene))]

In [137]:
#TEs

In [138]:
df_dat_tes = pd.read_csv(path_dat_tes + 'diffexpr-results.0.1.csv', index_col=False, sep=',', comment='#')
print(len(df_dat_tes.index))

21


In [139]:
df_dat_tes = df_dat_tes[['Gene','log2FoldChange']]
df_dat_tes.rename(columns={'Gene':'TE','log2FoldChange':'log2FoldChange_TE'}, inplace=True)

In [140]:
df_dat_tes['position_TE'] = df_dat_tes.TE.str.split(' _ ').str[1]
df_dat_tes['TE'] = df_dat_tes.TE.str.split(' _ ').str[0]

In [141]:
df_dat_tes.sample(3)

,TE,log2FoldChange_TE,position_TE
14,XXX_Taes_Ladon_AF325198-1,2.966002,7B:616772521-616772710
19,Copia-5_TA-I,6.003682,4D:393818669-393826225
2,ANGELA1_TM_I,4.769587,1B:514285370-514289839


In [142]:
df_dat_tes['seqname'] = df_dat_tes.position_TE.str.split(':').str[0]
df_dat_tes['start'] = df_dat_tes.position_TE.str.split('-').str[0]
df_dat_tes['start'] = df_dat_tes.start.str.split(':').str[1]
df_dat_tes['end'] = df_dat_tes.position_TE.str.split('-').str[1]


In [143]:
df_dat_tes.sample(3)

,TE,log2FoldChange_TE,position_TE,seqname,start,end
14,XXX_Taes_Ladon_AF325198-1,2.966002,7B:616772521-616772710,7B,616772521,616772710
4,Copia-2_TA-I,7.109906,3B:441907282-441916497,3B,441907282,441916497
17,RLG_Taes_Nusif_3BctgE-7,6.230046,3B:291988255-291991584,3B,291988255,291991584


In [144]:
#merge TE and DEGs by position
df_res.sample(3)

,gene,log2FoldChange,seqname,start,end,promoter,log2FoldChange_promoter,sRNA_gene,log2FoldChange_gene
339,TraesCS6A02G028100,5.954220,6A,14373125,14377288,NaN,NaN,NaN,NaN
819,TraesCS1D02G275200,2.236091,1D,370917273,370918280,NaN,NaN,NaN,NaN
942,TraesCS4D02G236400,1.046008,4D,398293345,398296122,NaN,NaN,NaN,NaN


In [145]:
df_res['TE'] = ''
df_res['log2FoldChange_TE'] = ''
df_res['position_TE'] = ''

In [174]:
howfar = 2000
for k,v in df_res.iterrows():
    for k_te, v_te in df_dat_tes.iterrows():
        if v_te.seqname == v.seqname:
            te_start = int(min(v_te.start,v_te.end))
            te_end = int(max(v_te.start,v_te.end))
            
            gene_start = int(min(v.start,v.end))
            gene_end = int(max(v.start,v.end))
            gene_start = int(max(gene_start - howfar, 0))
            gene_end += howfar
            if gene_end >= te_start and gene_start <= te_end:
                df_res.at[k, 'TE'] = v_te.TE
                df_res.at[k, 'log2FoldChange_TE'] = v_te.log2FoldChange_TE
                df_res.at[k, 'position_TE'] = v_te.position_TE

In [159]:
df_res.sample(5)

,gene,log2FoldChange,seqname,start,end,promoter,log2FoldChange_promoter,sRNA_gene,log2FoldChange_gene,TE,log2FoldChange_TE,position_TE
664,TraesCS7B02G050500,3.495037,7B,52347658,52350365,NaN,NaN,NaN,NaN,,,
2970,TraesCS2B02G411900,-0.648452,2B,586118399,586120605,NaN,NaN,NaN,NaN,,,
420,TraesCS3B02G011600,5.645763,3B,5199156,5201287,NaN,NaN,NaN,NaN,,,
453,TraesCS4B02G267200,-0.705791,4B,539965615,539970295,NaN,NaN,NaN,NaN,,,
1057,TraesCS3D02G401400,-1.442054,3D,515965563,515968953,NaN,NaN,NaN,NaN,,,


In [161]:
df_res[df_res.position_TE != '']

,gene,log2FoldChange,seqname,start,end,promoter,log2FoldChange_promoter,sRNA_gene,log2FoldChange_gene,TE,log2FoldChange_TE,position_TE


In [162]:
df_res.to_csv(file_output, index=None, )